# Semi-Signed Distance Field for neural representation of open surfaces
This Jupyter Notebook serves as a way to display the results from training a neural network to represent geometric shapes.

You can visualize a 2D cross section of a particular shape and see how to access the latent vectors

#### Libraries

In [ ]:
# Geometric libraries
import igl
from pygel3d import hmesh, jupyter_display as jd
jd.set_export_mode(True)

# numpy libraries
import numpy as np
import math
import random

# Deep learning libraries
import torch
import torch.nn as nn
import torch.nn.functional as Func
import torch.optim as optim
import torch.nn.init as init
from torch.nn.parameter import Parameter

# Libraries for environmetn
import os
from dotenv import load_dotenv

import matplotlib.pyplot as plt

In [ ]:
os.sys.path.insert(1,os.path.join(os.getcwd(),"..",".."))

#### External functions

In [ ]:
from utils.general import parse_cfg_file
from utils.neural_network import data
from utils.neural_network import Net   

In [ ]:
assert os.path.exists(os.path.join(os.getcwd(),"..","..",".env")), \
        "Please create an .env file with appropriate directories and specification of configuration file"
load_dotenv()
#--------------------------- 
# Environment
#---------------------------

# Data sets 
training_mesh_dir = os.path.join(os.getcwd(),os.getenv('mesh_train_dir'))
test_mesh_dir = os.path.join(os.getcwd(),os.getenv('mesh_test_dir'))
shape_completion_mesh_dir = os.path.join(os.getcwd(),os.getenv('mesh_shape_completion_dir'))

# Model dir
train_model_dir = os.path.join(os.getcwd(),os.getenv('gwn_model_train_dir'))
test_model_dir = os.path.join(os.getcwd(),os.getenv('gwn_model_test_dir'))
shape_completion_model_dir = os.path.join(os.getcwd(),os.getenv('gwn_model_shape_completion_dir'))

# Interpolation dir
interpolation_dir = os.path.join(os.getcwd(),os.getenv('interpolation_dir'))
shape_reconsruction_dir = os.path.join(os.getcwd(),"shape_reconstruction")

# Configuration file
cfg_file = os.getenv('cfg_file')
assert os.path.exists(cfg_file), \
    "Make sure you have the config file (.yaml) in the cfgs folder and set correct path in .env file"
cfg = parse_cfg_file(cfg_file)

### Use GPU or CPU

In [ ]:
if (cfg.use_GPU):
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
else:
    device = 'cpu'

## Load data and network

#### Training data - Ground truth meshes and latent vectors

In [ ]:
training_data = data(training_mesh_dir, train_model_dir, cfg.model_train.name, device)
print("Number of training files:", len(training_data))

#### Test data - Ground truth meshes and latent vectors

In [ ]:
test_data = data(test_mesh_dir, test_model_dir, cfg.model_test.name, device)
print("Number of test files:", len(test_data))

#### Shape completion

In [ ]:
shape_completion_data = data(shape_completion_mesh_dir, shape_completion_model_dir, 
                             cfg.model_shape_completion.name, device, test_mesh_dir)
print("Number of shape completion files:", len(shape_completion_data))

#### Neural Network

In [ ]:
net = Net(cfg.latent_vector.size + cfg.network.size_point, 
                cfg.network.num_hidden_units, cfg.network.single_output, device)

checkpoint = torch.load(os.path.join(train_model_dir,cfg.model_train.name),map_location=device)
net.load_state_dict(checkpoint['net_state_dict'])
net = net.to(device)
net.eval()

## Cross Sectional result

In [ ]:
'''
Training data
'''
#mesh, Faces, latent_vector = training_data["your_file.obj"] # Replace .obj file with name of own file

'''
Test data
'''
mesh, Faces, latent_vector = test_data["your_file.obj"]

In [ ]:
pos = mesh.positions()
epsilon = 0.1
bbox = hmesh.bbox(mesh)+epsilon*np.array([[-1,-1,-1],[1,1,1]])
resolution = 500
Y = np.linspace(bbox[0][1], bbox[1][1], resolution)
Z = np.linspace(bbox[0][2], bbox[1][2], resolution)
P = np.array([ np.array([0.0, Y[idx[0]], Z[idx[1]]]) for idx in np.ndindex((resolution,resolution))])

In [ ]:
GWN = 2*np.array(igl.fast_winding_number_for_meshes(np.array(pos),np.array(Faces),np.array(P)))\
    -np.ones([1,resolution*resolution])
GWN = GWN.reshape((resolution,resolution))

In [ ]:
net_input = torch.cat((latent_vector.repeat(len(P),1).detach(),torch.from_numpy(P).float().to(device)),1)

net_output = net(net_input)
net_GWN = net_output.detach().numpy()
net_GWN = net_GWN.reshape((resolution,resolution))

In [ ]:
plt.figure()
f, axarr = plt.subplots(nrows=1, ncols=2, figsize=(20, 20), constrained_layout=True)

# Plot 1 - left
ax = axarr[0]
levels = [0]
cs = ax.contour(GWN, levels,colors=['red'])
im3 = ax.imshow(GWN, interpolation='bilinear', origin='lower')
ax.set_xlabel('y',fontsize=25)
ax.set_ylabel('z',fontsize=25)
f.colorbar(im3,ax = axarr[0],fraction=0.05, pad=0.06).ax.tick_params(labelsize=40)
ax.set_title("Ground Truth GWN, x = 0.0", size=30)

# Plot 2 - right 
ax = axarr[1]
levels = [0]
cs = ax.contour(net_GWN, levels,colors=['red'] )
im3 = ax.imshow(net_GWN, interpolation='bilinear', origin='lower')
ax.set_xlabel('y',fontsize=25)
ax.set_ylabel('z',fontsize=25)
f.colorbar(im3,ax = axarr[1],fraction=0.05, pad=0.06).ax.tick_params(labelsize=40)
ax.set_title("Network GWN, x = 0.0", size=30)

### Latent Vectors 

In [ ]:
training_data.latent_vectors.detach().cpu().numpy()

In [ ]:
test_data.latent_vectors.detach().cpu().numpy()

In [ ]:
shape_completion_data.latent_vectors.detach().cpu().numpy()